In [ ]:
import pandas as pd
import openpyxl as op
from openpyxl import load_workbook
from openpyxl.utils.cell import get_column_letter
from openpyxl.styles import Border, Side, Font, Alignment, numbers
import numpy as np
from datetime import datetime
startTime = datetime.now()
pd.set_option('display.max_columns', None)
accounting_format = u'_($* #,##0.00_);[Red]_($* (#,##0.00);_($* -_0_0_);_(@'
percentage_format = numbers.FORMAT_PERCENTAGE_00

In [ ]:
month = (input("Enter commission month: ")).capitalize()
month

In [ ]:
rep_detail = pd.read_excel('rep_detail_input.xlsx') #REP DETAIL LIST

In [ ]:
sales_data = pd.read_excel('filename.xlsx', keep_default_na = False, parse_dates = ['Invc Date']).copy() #MONTHLY SALES DATA, keep blanks, use a copy
sales_data['Invc Date'] = sales_data['Invc Date'].dt.date

In [ ]:
for i in range(len(rep_detail.index)):
    rep_code_list = str(rep_detail['Rep Code'][i]).split(' ')
    
    path = rep_detail['Path'][i]
    
    sales_data_copy = sales_data
    rep_df = sales_data_copy.loc[sales_data_copy['Rep'].isin(rep_code_list)]
    
    wb = load_workbook(path)

    writer = pd.ExcelWriter(path, engine = 'openpyxl')
    writer.book = wb
    
    rep_df.to_excel(writer, index = False, sheet_name = month)
    
    sheet = wb[month]
    
    ##find column number for the needed columns
    unitcost_colnum = rep_df.columns.get_loc('Unit Cost')+1
    stdunitcost_colnum = unitcost_colnum+1
    price_colnum = unitcost_colnum+2
    net_colnum = unitcost_colnum+3
    grosspercent_colnum = unitcost_colnum+4
    grossdollar_colnum = unitcost_colnum+5
    
    ## format each cell in needed columns to be accounting format
    for i in range(len(rep_df['Unit Cost'])):
        cell = sheet.cell(row = i+2, column = unitcost_colnum)
        cell.number_format = accounting_format
    
    for i in range(len(rep_df['Std Cost Ext'])):
        cell = sheet.cell(row = i+2, column = stdunitcost_colnum)
        cell.number_format = accounting_format
        
    for i in range(len(rep_df['Price FDP'])):
        cell = sheet.cell(row = i+2, column = price_colnum)
        cell.number_format = accounting_format
        
    for i in range(len(rep_df['Net Amount'])):
        cell = sheet.cell(row = i+2, column = net_colnum)
        cell.number_format = accounting_format
        
    for i in range(len(rep_df['Gross Margin off Std Cost'])):
        cell = sheet.cell(row = i+2, column = grossdollar_colnum)
        cell.number_format = accounting_format
    
    ## format each cell in needed columns to be percentage format
    for i in range(len(rep_df['GP % off Standard Cost'])):
        cell = sheet.cell(row = i+2, column = grosspercent_colnum)
        cell.number_format = percentage_format
        
    column_count = len(rep_df.columns)
    column_number = 1
    sheet = wb[month]
    
    while column_number <= column_count:
        max = len(str(rep_df.columns.values[column_number-1]))
        col = list(sales_data_copy[rep_df.columns.values[column_number-1]])
    
        for cell in col:
            if len(str(cell)) > max:
                max = len(str(cell))
        
        sheet.column_dimensions[get_column_letter(column_number)].width = max+3
        column_number += 1
    
    
    ## pivot table of customer no, customer name, and sum of net amount
    piv = rep_df.groupby(['Cust No', 'Customer Name']).sum()['Net Amount']
    piv.to_excel(writer, sheet_name = 'Pivot')

    writer.save()
    writer.close()
    wb.save(path)